In [1]:
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('log').getOrCreate()

In [3]:
%fs ls /FileStore/tables

path,name,size
dbfs:/FileStore/tables/ContainsNull.csv,ContainsNull.csv,61
dbfs:/FileStore/tables/Ecommerce_Customers.csv,Ecommerce_Customers.csv,86871
dbfs:/FileStore/tables/Spark_Essentials-5d27c.dbc,Spark_Essentials-5d27c.dbc,1414841
dbfs:/FileStore/tables/appl_stock.csv,appl_stock.csv,143130
dbfs:/FileStore/tables/cogsley_clients.csv,cogsley_clients.csv,384219
dbfs:/FileStore/tables/cogsley_sales.csv,cogsley_sales.csv,2176442
dbfs:/FileStore/tables/cruise_ship_info.csv,cruise_ship_info.csv,8734
dbfs:/FileStore/tables/people.json,people.json,73
dbfs:/FileStore/tables/sales_info.csv,sales_info.csv,196
dbfs:/FileStore/tables/sample_libsvm_data.txt,sample_libsvm_data.txt,104736


In [4]:
path = '/FileStore/tables/sample_libsvm_data.txt'

In [5]:
df = spark.read.format('libsvm').load(path)

In [6]:
df.show(5)

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0|(692,[127,128,129...|
 1.0|(692,[158,159,160...|
 1.0|(692,[124,125,126...|
 1.0|(692,[152,153,154...|
 1.0|(692,[151,152,153...|
+-----+--------------------+
only showing top 5 rows

In [7]:
from pyspark.ml.classification import LogisticRegression

In [8]:
log = LogisticRegression(featuresCol='features',labelCol='label',predictionCol='prediction')

In [9]:
log_model = log.fit(df)

In [10]:
log_summary = log_model.summary

In [11]:
log_summary.predictions.printSchema()

root
-- label: double (nullable = true)
-- features: vector (nullable = true)
-- rawPrediction: vector (nullable = true)
-- probability: vector (nullable = true)
-- prediction: double (nullable = false)

In [12]:
log_summary.roc.show()

+---+--------------------+
FPR| TPR|
+---+--------------------+
0.0| 0.0|
0.0|0.017543859649122806|
0.0| 0.03508771929824561|
0.0| 0.05263157894736842|
0.0| 0.07017543859649122|
0.0| 0.08771929824561403|
0.0| 0.10526315789473684|
0.0| 0.12280701754385964|
0.0| 0.14035087719298245|
0.0| 0.15789473684210525|
0.0| 0.17543859649122806|
0.0| 0.19298245614035087|
0.0| 0.21052631578947367|
0.0| 0.22807017543859648|
0.0| 0.24561403508771928|
0.0| 0.2631578947368421|
0.0| 0.2807017543859649|
0.0| 0.2982456140350877|
0.0| 0.3157894736842105|
0.0| 0.3333333333333333|
+---+--------------------+
only showing top 20 rows

In [13]:
log_summary.predictions.show(5)

+-----+--------------------+--------------------+--------------------+----------+
label| features| rawPrediction| probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
 0.0|(692,[127,128,129...|[19.8534775947479...|[0.99999999761359...| 0.0|
 1.0|(692,[158,159,160...|[-20.377398194908...|[1.41321555110992...| 1.0|
 1.0|(692,[124,125,126...|[-27.401459284891...|[1.25804865126989...| 1.0|
 1.0|(692,[152,153,154...|[-18.862741612668...|[6.42710509170283...| 1.0|
 1.0|(692,[151,152,153...|[-20.483011833009...|[1.27157209200603...| 1.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows

In [14]:
log_summary.areaUnderROC

Out[ 20 ]: 1.0

In [15]:
# redoing

In [16]:
train,test = df.randomSplit([0.7,0.3])

In [17]:
log = LogisticRegression()

In [18]:
log_model = log.fit(train)

In [19]:
pred = log_model.evaluate(test)

In [20]:
print(pred.areaUnderROC)
print(pred.accuracy)
print(pred.predictions.show())


1.0
1.0
+-----+--------------------+--------------------+--------------------+----------+
label| features| rawPrediction| probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
 0.0|(692,[98,99,100,1...|[28.3649623663887...|[0.99999999999951...| 0.0|
 0.0|(692,[122,123,148...|[19.5773637502177...|[0.99999999685472...| 0.0|
 0.0|(692,[124,125,126...|[31.7994732631399...|[0.99999999999998...| 0.0|
 0.0|(692,[124,125,126...|[18.8754154050282...|[0.99999999365383...| 0.0|
 0.0|(692,[125,126,127...|[20.5246911199065...|[0.99999999878033...| 0.0|
 0.0|(692,[126,127,128...|[23.1932421177466...|[0.99999999991541...| 0.0|
 0.0|(692,[129,130,131...|[15.7628432843055...|[0.99999985734595...| 0.0|
 0.0|(692,[151,152,153...|[28.3908373056244...|[0.99999999999953...| 0.0|
 0.0|(692,[152,153,154...|[21.6764373132277...|[0.99999999961448...| 0.0|
 0.0|(692,[152,153,154...|[28.5611435970026...|[0.99999999999960...| 0.0|
 0.0|(692,[153,154,155...|[26.1192389951064...|[0.99999999999546...| 0.0|
 0.0|(692,[154,155,156...|[11.3783803254054...|[0.99998855996789...| 0.0|
 0.0|(692,[154,155,156...|[18.9834955236236...|[0.99999999430396...| 0.0|
 0.0|(692,[181,182,183...|[20.1726455617679...|[0.99999999826567...| 0.0|
 1.0|(692,[124,125,126...|[-24.175509869557...|[3.16744780044853...| 1.0|
 1.0|(692,[125,126,127...|[-19.574532798775...|[3.15419412355847...| 1.0|
 1.0|(692,[126,127,128...|[-32.015521642823...|[1.24691145666047...| 1.0|
 1.0|(692,[127,128,129...|[-26.027046222808...|[4.97275937917453...| 1.0|
 1.0|(692,[127,128,129...|[-31.286781708004...|[2.58418349116632...| 1.0|
 1.0|(692,[128,129,130...|[-26.408311693302...|[3.39637750055193...| 1.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 20 rows

None

In [21]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [22]:
evaluator = BinaryClassificationEvaluator()

In [23]:
my_final_roc = evaluator.evaluate(pred.predictions)

In [24]:
my_final_roc

Out[ 46 ]: 1.0